## Elo multi-player rating method

This is the Elo method for a multi-competitor format. The implementation is based on the algorithm mentioned at the end of the article: 

https://fivethirtyeight.com/features/formula-one-racing/

Simulating Elo multi-player on International Informatics Olympiad contests between 2011 and 2022

In [1]:
import math 
  
# Function to calculate the Probability 
def Probability(rating1, rating2): 
  
    return 1.0 * 1.0 / (1 + 1.0 * math.pow(10, 1.0 * (rating1 - rating2) / 1000)) 
  
# Function to calculate Elo rating 
# K is a constant. 
# Player A wins over Player B.  
# tie = true if tie, false otherwise
def EloRating(Ra, Rb, K, tie): 
    
    # To calculate the Winning 
    # Probability of Player B 
    Pb = Probability(Ra, Rb) 
  
    # To calculate the Winning 
    # Probability of Player A 
    Pa = Probability(Rb, Ra) 
  
    # Updating the Elo Ratings 
    if tie:
       Ra = Ra + K * (1/2 - Pa) 
       Rb = Rb + K * (1/2 - Pb) 
    else:        
       Ra = Ra + K * (1 - Pa) 
       Rb = Rb + K * (0 - Pb) 
    
    return Ra, Rb

### Loading contests datasets

In [2]:
import pandas as pd
import os.path

#directory of the dataset
filedir = "PerOthers" 
#type of the dataset (For instance C for whole contests, AH for AdHoc problems, and IN for interactive problems)
filetype = "OT" 
#filename {directory}/IOI{year}{type}.csv Ex: PerWholeContest/IOI2022C.csv
filename = "{}/IOI{}{}.csv"

#dictionary of contests data => contests[year] = pandas DataFrame
contests = dict()

#read from firstYear contest through lastYear contest
firstYear = 2011
lastYear = 2022

#list of years where specific categories exist
years = []

#Reading csv files into DataFrames 
for year in range(firstYear, lastYear + 1):
    file = filename.format(filedir, year, filetype)
    
    #check if file exists (some years don't comprise problems of some categories)
    if os.path.isfile(file):
        years.append(year)
        contests[year] = pd.read_csv(file, encoding='unicode_escape')

### Simulate Elo ratings

In [3]:
#dictionary of ratings => ratings[nation] = rating
ratings = dict()

#dictionary of ratings history over years
#where => ratingsHistory[nation] = {year1: rating_year1, year2: rating_year2...}
#purpose: extracting results 
ratingsHistory = dict()

#simulating the multiplayer rating over years
for year in years:

    noOfNations = len(contests[year])

    #intializing nations with no prior rating to TrueSkill initial rating
    for i in range(noOfNations):
        nation = contests[year].loc[i,"Country"]

        if nation not in ratings:
            ratings[nation] = 1500 #considering 1500 as intial Elo ratings 
            ratingsHistory[nation] = {0: 1500} #0 represents initial rating

    for i in range(noOfNations):
        team1Nation = contests[year].loc[i,"Country"]
        team1Rank = contests[year].loc[i,"Rank"]

        for j in range(i+1,noOfNations):
            team2Nation = contests[year].loc[j,"Country"]
            team2Rank = contests[year].loc[j,"Rank"]
            if team1Rank < team2Rank: # team1 beats team2 due to lower rank
                team1Rating, team2Rating = EloRating(ratings[team1Nation], ratings[team2Nation], 4.4, False)
            elif team1Rank == team2Rank: # team1 ties team2 in rank
                team2Rating, team1Rating = EloRating(ratings[team2Nation], ratings[team1Nation], 4.4, True)
                
            ratings[team1Nation] = team1Rating
            ratings[team2Nation] = team2Rating

    #appending ratings after a year's contest to the ratingHistory
    for nation in ratings:
        if nation in contests[year].values:
            ratingsHistory[nation][year] = ratings[nation]

### Extracting results

In [4]:
#Extracting csv file for rating history of each nation
for nation in ratingsHistory:
    nationData = {"Year": [], "Rating": []}
    
    for year in ratingsHistory[nation]:
        nationData["Year"].append(year)
        nationData["Rating"].append(ratingsHistory[nation][year])
    
    nationHistory = pd.DataFrame(nationData)
    
    historyFileName = 'nationsRatingsChanges{}/elo/{}.csv'
    nationHistory.to_csv(historyFileName.format(filedir, nation), index=False)
    
    
#Extracting csv file for ratings and ranks for each year contest

#dictionary of contests TrueSkill ratings data => contestsRating[year] = pandas DataFrame
contestsRatings = dict()

for year in years:
    contestRatings = {"Country": [], "Rating": []}
    
    for nation in ratingsHistory:
        if year in ratingsHistory[nation]:
            contestRatings["Country"].append(nation)
            contestRatings["Rating"].append(ratingsHistory[nation][year])
    
    contestRatingsDF = pd.DataFrame(contestRatings)
    
    #sorting the DataFrame AND adding ranking column
    contestRatingsDF.sort_values(["Rating"], ascending=False, inplace=True)
    
    contestRanks = range(1, len(contestRatingsDF.index)+1)
    contestRatingsDF["Rank"] = contestRanks
    
    #Extracting CSV and appending contest DataFrame to contestsRatings
    
    contestsRatings[year] = contestRatingsDF
    contestFileName = 'contestRatings{}/elo/{}.csv'
    contestRatingsDF.to_csv(contestFileName.format(filedir, year), index=False)

### Calculating predective accuracies

Calculating the predectivity by comparing the rataings of each contest with the rankings of the following contest.

This is done through dividing the nations into combination of pairs and determine whether each pair is predicted correctly.

In [5]:
#storing predective accuracies for csv extraction 
predectiveAccuracies = {"Year": [], "Predective Accuracy": []}

print('\n\n************** Elo Predectivity **************\n')
print('===========================')
print('Year    Predictive Accuracy')
print('===========================')

for year in range(0, len(years) - 1):
    
    noOfCombinations = 0
    truePredections = 0
    
    noOfNationsInNextContest = len(contestsRatings[years[year+1]])
    
    currentContestDF = contestsRatings[years[year]]
    
    for i in range(noOfNationsInNextContest):
        
        team1Nation = contests[years[year + 1]].loc[i,"Country"]
        team1Rank = contests[years[year + 1]].loc[i,"Rank"]
        
        #if a nation participated in the following contest but did not participate in the current contest,
        #this code looks for the nearst avaliable rank for this nation.
        #if not found, the nation's rating is considered as the initial rating of Elo (1500).
        if team1Nation not in currentContestDF["Country"].values: 
            team1Rating = 1500
            
            for _year in ratingsHistory[team1Nation]:
                if _year < years[year]:
                    team1Rating = ratingsHistory[team1Nation][_year]
        else:
            team1Rating = ratingsHistory[team1Nation][years[year]]
            
            
        for j in range(i+1, noOfNationsInNextContest):
            
            team2Nation = contests[years[year + 1]].loc[j,"Country"]
            team2Rank = contests[years[year + 1]].loc[j,"Rank"]
            
            if team2Nation not in currentContestDF["Country"].values: 
                team2Rating = 1500
                
                for _year in ratingsHistory[team2Nation]:
                    if _year < years[year]:
                        team2Rating = ratingsHistory[team2Nation][_year]
            else:
                team2Rating = ratingsHistory[team2Nation][years[year]]
            
            if(team1Nation == team2Nation):
                continue
                
            noOfCombinations += 1
            
            if team1Rating > team2Rating and team1Rank < team2Rank:
                truePredections += 1
            elif team1Rating < team2Rating and team1Rank > team2Rank:
                truePredections += 1
            elif team1Rating == team2Rating and team1Rank == team2Rank:
                truePredections += 1
    
    
    predectivity = round((truePredections / noOfCombinations) * 100, 4)
    
    print(f'{years[year+1]}       {predectivity}%')
    
    predectiveAccuracies["Year"].append(years[year + 1])
    predectiveAccuracies["Predective Accuracy"].append(predectivity)


#Extracting accuracies to csv
predectiveDF = pd.DataFrame(predectiveAccuracies)
    
predectiveFileName = 'predectiveAccuracies{}/elo.csv'
predectiveDF.to_csv(predectiveFileName.format(filedir), index=False)



************** Elo Predectivity **************

Year    Predictive Accuracy
2012       64.5062%
2013       72.5738%
2016       77.5309%
2019       71.7455%
2020       78.6955%
2021       72.4974%
2022       73.6829%


85.2934